In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

# Load the Iris dataset
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['species'] = iris.target

# Encode the target labels (species)
label_encoder = LabelEncoder()
data['species'] = label_encoder.fit_transform(data['species'])

# Separate features and target
X = data.drop('species', axis=1).values
y = data['species'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [10]:
class IrisNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)  # First hidden layer
        self.fc2 = nn.Linear(16, 8)          # Second hidden layer
        self.fc3 = nn.Linear(8, output_dim)  # Output layer (3 classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x  # No softmax here, as CrossEntropyLoss includes softmax


In [11]:
# Initialize the model
model = IrisNet(X_train.shape[1], len(set(y)))  # 4 features, 3 classes

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss includes softmax
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Clear gradients
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    _, predicted_labels = torch.max(outputs, 1)
    correct = (predicted_labels == y_train_tensor).sum().item()
    train_accuracy = correct / y_train_tensor.size(0)
    
    if (epoch + 1) % 10 == 0:  # Print every 10 epochs
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {train_accuracy:.4f}')


Epoch [10/1000], Loss: 1.0757, Accuracy: 0.3667
Epoch [20/1000], Loss: 1.0404, Accuracy: 0.7333
Epoch [30/1000], Loss: 1.0046, Accuracy: 0.7917
Epoch [40/1000], Loss: 0.9660, Accuracy: 0.7917
Epoch [50/1000], Loss: 0.9229, Accuracy: 0.7917
Epoch [60/1000], Loss: 0.8747, Accuracy: 0.7917
Epoch [70/1000], Loss: 0.8225, Accuracy: 0.8167
Epoch [80/1000], Loss: 0.7671, Accuracy: 0.8083
Epoch [90/1000], Loss: 0.7110, Accuracy: 0.8083
Epoch [100/1000], Loss: 0.6570, Accuracy: 0.8250
Epoch [110/1000], Loss: 0.6072, Accuracy: 0.8250
Epoch [120/1000], Loss: 0.5622, Accuracy: 0.8250
Epoch [130/1000], Loss: 0.5216, Accuracy: 0.8333
Epoch [140/1000], Loss: 0.4845, Accuracy: 0.8417
Epoch [150/1000], Loss: 0.4504, Accuracy: 0.8417
Epoch [160/1000], Loss: 0.4189, Accuracy: 0.8500
Epoch [170/1000], Loss: 0.3896, Accuracy: 0.8667
Epoch [180/1000], Loss: 0.3623, Accuracy: 0.8833
Epoch [190/1000], Loss: 0.3370, Accuracy: 0.9000
Epoch [200/1000], Loss: 0.3136, Accuracy: 0.9167
Epoch [210/1000], Loss: 0.291

In [12]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation during inference
    test_outputs = model(X_test_tensor)
    _, predicted_labels = torch.max(test_outputs, 1)
    correct = (predicted_labels == y_test_tensor).sum().item()
    test_accuracy = correct / y_test_tensor.size(0)

print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 1.0000


In [13]:
# Save the trained model
# torch.save(model.state_dict(), 'iris_model.pth')
scripted_model = torch.jit.script(model)  # or torch.jit.trace(model, example_input)
scripted_model.save("temp_model.pt")
print("Model saved successfully!")


Model saved successfully!


In [14]:
first_row = data.iloc[0, :-1].values

model(torch.tensor(first_row, dtype=torch.float32))
# Load the model


tensor([-6.7720,  7.4369, -2.9274], grad_fn=<ViewBackward0>)

In [15]:
first_row


array([5.1, 3.5, 1.4, 0.2])